# Header 


In [48]:
import pandas as pd
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [23]:
df_simple = pd.read_csv("simpleData.csv")
df_simple.head()

,LoanId,OrderId,PortfolioId,NoteId,Invested,InterestRate,Grade,LoanType,Term,PrincipalRemaining,...,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,PURCHASE_PRINCIPAL,PURCHASE_INTEREST,PRINCIPAL_RECEIVED,INTEREST_RECEIVED
0,62127260,133566975,167112082,143275084,25,18.99%,D3,Personal,36,2.679266,...,2.0,8054.0,61.0,1.0,3.0,3.0,25,0,22.320734,8.001124
1,70904954,133566975,167112082,143275086,25,7.99%,A5,Personal,36,1.550690,...,6.0,5113.0,41.0,4.0,0.0,2.0,25,0,23.449310,3.188582
2,87255474,127709459,167112084,138178962,25,7.59%,A3,Personal,36,0.000000,...,3.0,3572.0,57.0,2.0,0.0,2.0,25,0,25.000000,2.844445
3,88535138,127749223,167112084,138149319,25,16.99%,D1,Personal,36,0.000000,...,6.0,6340.0,54.0,0.0,0.0,0.0,25,0,25.000000,7.035707
4,88593479,127709459,167112082,138178963,25,16.99%,D1,Personal,36,0.024581,...,0.0,1687.0,70.0,2.0,0.0,1.0,25,0,24.975419,7.060581


In [24]:
allnames = set(df_simple.columns)
allnames

{'1st Month Payment',
 'Accrual',
 'Grade',
 'INTEREST_RECEIVED',
 'InterestRate',
 'Invested',
 'Loan Issue Date',
 'LoanId',
 'LoanStatusDate',
 'LoanType',
 'NextPaymentDate',
 'Note Issue Date',
 'NoteId',
 'OrderDate',
 'OrderId',
 'PRINCIPAL_RECEIVED',
 'PURCHASE_INTEREST',
 'PURCHASE_PRINCIPAL',
 'PaymentsReceivedToDate',
 'Pending Interest',
 'Pending Principal Payment',
 'PortfolioId',
 'PrincipalRemaining',
 'Status',
 'Term',
 'Trend',
 'acc_now_delinq',
 'acc_open_past_24mths',
 'accept_d',
 'all_util',
 'annual_inc',
 'annual_inc_joint',
 'application_type',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'collection_recovery_fee',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'delinq_amnt',
 'desc',
 'dti',
 'dti_joint',
 'emp_length',
 'emp_title',
 'exp_d',
 'fico_range_high',
 'fico_range_low',
 'home_ownership',
 'il_util',
 'inq_fi',
 'inq_last_12m',
 'inq_last_6mths',
 'installment',
 'is_inc_v',
 'last_credit_pull_d',
 'last_fico_rang

Turn the credit score range into estimated credit score

In [26]:
df_simple['credit_score']=(df_simple['last_fico_range_high']+df_simple['last_fico_range_low'])/2.
df_simple.drop(columns = ['last_fico_range_high','last_fico_range_low'])
df_simple.head()

,LoanId,OrderId,PortfolioId,NoteId,Invested,InterestRate,Grade,LoanType,Term,PrincipalRemaining,...,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,PURCHASE_PRINCIPAL,PURCHASE_INTEREST,PRINCIPAL_RECEIVED,INTEREST_RECEIVED,credit_score
0,62127260,133566975,167112082,143275084,25,18.99%,D3,Personal,36,2.679266,...,8054.0,61.0,1.0,3.0,3.0,25,0,22.320734,8.001124,582.0
1,70904954,133566975,167112082,143275086,25,7.99%,A5,Personal,36,1.550690,...,5113.0,41.0,4.0,0.0,2.0,25,0,23.449310,3.188582,722.0
2,87255474,127709459,167112084,138178962,25,7.59%,A3,Personal,36,0.000000,...,3572.0,57.0,2.0,0.0,2.0,25,0,25.000000,2.844445,677.0
3,88535138,127749223,167112084,138149319,25,16.99%,D1,Personal,36,0.000000,...,6340.0,54.0,0.0,0.0,0.0,25,0,25.000000,7.035707,757.0
4,88593479,127709459,167112082,138178963,25,16.99%,D1,Personal,36,0.024581,...,1687.0,70.0,2.0,0.0,1.0,25,0,24.975419,7.060581,612.0


Calculate the correlation between credit score and sucessful full payment for the full paid loans.  

In [46]:
status_groups = df_simple.groupby(['Status'])
for i in status_groups.groups:
    print(i)
paid_fico_hist = status_groups.get_group('Fully Paid')['credit_score'].hist()


Charged Off
Current
Default
Fully Paid
In Grace Period
Issued
Issuing
Late (16-30 days)
Late (31-120 days)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
default_fico_hist = status_groups.get_group('Default')['credit_score'].hist()

Calculate relative payment and full payment. Where full payment is boolean and relative payment is real valued. Note people can pay more than their installment.

In [28]:
test_df = df_simple[['installment','last_pymnt_amnt']]
test_df.head(15)

,installment,last_pymnt_amnt
0,1099.53,1099.53
1,438.65,438.65
2,252.30,2216.33
3,883.18,882.86
4,534.72,534.72
5,185.24,185.05
6,399.54,2830.13
7,549.77,549.77
8,585.08,11861.31
9,674.15,16442.77


In [ ]:
test_df['last_payment_full']